### Used for understanding the Mock Package

In [1]:
from unittest.mock import Mock

mock = Mock()

mock

<Mock id='139837561060496'>

In [ ]:
def add(t):
    return 50 + t

result = add(mock)

In [3]:
mock.add()

<Mock name='mock.add()' id='139837612218896'>

In [4]:
mock.value

<Mock name='mock.value' id='139837318557328'>

Unlike the real dumps(), this mocked method requires no arguments. In fact, it will accept any arguments that you pass to it.

The return value of dumps() is also a Mock. The capability of Mock to recursively define other mocks allows for you to use mocks in complex situations:

In [5]:
json = Mock()

json.loads('{"key":"val"}').get('k')

<Mock name='mock.loads().get()' id='139837320333264'>

**Mock instances store data on how you used them.** 

For instance, you can see if you called a method, how you called the method, and so on. There are two main ways to use this information.

**First, you can assert that your program used an object as you expected:**

In [8]:
json.loads.assert_called()
json.loads.assert_called_once()
json.loads.assert_called_with('{"key":"val"}')

**Second, you can view special attributes to understand how your application used an object:**

In [9]:
json.loads.call_args

call('{"key":"val"}')

In [10]:
json.loads.call_count

1

In [11]:
json.method_calls

[call.loads('{"key":"val"}')]

In [12]:
json.loads.call_args_list

[call('{"key":"val"}')]

In [14]:
from datetime import datetime

def is_weekday():
    today = datetime.today()
    return (0 <= today.weekday() < 5)

assert is_weekday()

AssertionError: 

**writing tests, it is important to ensure that the results are predictable. You can use Mock to eliminate uncertainty from your code during testing**

In above case the tests will fail depending on day it is run, which is un-predictable

In [16]:
tuesday = datetime(year=2023,month=9,day=26)
saturday = datetime(year=2023,month=9,day=23)

In [24]:
tuesday

datetime.datetime(2023, 9, 26, 0, 0)

In [22]:
# mocking datetime itself to control date
datetime = Mock()

In [18]:
import datetime
def is_weekday():
    today = datetime.datetime.today()
    return (0 <= today.weekday() <5)

In [23]:
datetime.today.return_value = tuesday

assert is_weekday()

TypeError: '<=' not supported between instances of 'int' and 'Mock'

In [25]:
datetime.datetime.today.return_value = tuesday

In [26]:
assert is_weekday()

In [27]:
datetime.datetime.today.return_value = saturday

assert is_weekday()

AssertionError: 

today() is a mocked method. You’ve removed the inconsistency by assigning a specific day to the mock’s .return_value.

In [29]:
from requests.exceptions import Timeout
from unittest.mock import Mock
from unittest import TestCase

# Mock requests to control its behaviour

requests = Mock()

def get_holidays():
    r = requests.get('http://localhost/api/holidays')

    if r.status_code == 200:
        return r.json()

    return None

class TestCalendar(TestCase):

    def test_get_holidays_timeout(self):

        requests.get.side_effect = Timeout

        with self.assertRaises(Timeout):
            get_holidays()

In [30]:
class TestCalendar(TestCase):

    def log_request(self, url):
        # Log a fake request for test output purposes
        print(f'Making a request to {url}.')
        print('Request received!')

        # Create a new Mock to imitate a Response
        response_mock = Mock()
        response_mock.status_code = 200
        response_mock.json.return_value = {
            '12/25': 'Christmas',
            '7/4': 'Independence Day',
        }
        return response_mock

    def test_get_holidays_logging(self):
        # Test a successful, logged request

        requests.get.side_effect = self.log_request
        #the above call is faked with the log_request

        assert get_holidays()['12/25'] == 'Christmas'
        

### Configuring Mock

You can configure a Mock to set up some of the object’s behaviors. Some configurable members include .side_effect, .return_value, and .name. You configure a Mock when you create one or when you use .configure_mock().

In [31]:
m = Mock(name='trial mock')
n = Mock(side_effect=Exception)
x = Mock(return_value=9779)

In [41]:
x.return_value

9779

In [42]:
# Verbose, old Mock
response_mock = Mock()
response_mock.json.return_value = {
    '12/25': 'Christmas',
    '7/4': 'Independence Day',
}

# Shiny, new .configure_mock()
holidays = {'12/25': 'Christmas', '7/4': 'Independence Day'}
response_mock = Mock(**{'json.return_value': holidays})

###Monkey patching is the replacement of one object with another at runtime. 

Some reasons why you might prefer a context manager include the following:

You only want to mock an object for a part of the test scope.

You are already using too many decorators or parameters, which hurts your test’s readability.

In [43]:
from unittest.mock import patch

class TestCalendar(TestCase):
    def test_get_holidays_timeout(self):
        with patch('my_calendar.requests') as mock_requests:
            mock_requests.get.side_effect = Timeout
            with self.assertRaises(Timeout):
                get_holidays()
                mock_requests.get.assert_called_once()

Patching an Object’s Attributes

Let’s say you only want to mock one method of an object instead of the entire object. You can do so by using patch.object().

In [44]:
class TestCalendar(TestCase):
    
    @patch.object(requests, 'get', 
                  side_effect=requests.exceptions.Timeout)
    def test_get_holidays_timeout(self, mock_requests):
            with self.assertRaises(requests.exceptions.Timeout):
                get_holidays()

Learning how to use patch() is critical to mocking objects in other modules. However, sometimes it’s not obvious what the target object’s path is.

In [45]:
calendar = Mock(spec=['is_weekday','get_holidays'])

In [46]:
calendar.is_weekday()

<Mock name='mock.is_weekday()' id='139837318493904'>

In [47]:
calendar.create_event()

AttributeError: Mock object has no attribute 'create_event'

In [1]:
import my_functions

from unittest.mock import Mock

cale = Mock(spec=my_functions)

In [3]:
cale.create_event()

AttributeError: Mock object has no attribute 'create_event'

In [4]:
from unittest.mock import create_autospec

cale = create_autospec(my_functions)

In [6]:
cale.get_holidays()

<MagicMock name='mock.get_holidays()' id='139740857034192'>

In [7]:
import my_functions

from unittest.mock import patch

with patch('__main__.my_functions',autospec=True) as cale:
    cale.get_holidays()
    cale.is_weekend()

AttributeError: Mock object has no attribute 'is_weekend'